In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt
from typing import Dict, Any

# Hugging Face imports
from transformers import pipeline
import torch

class AdvancedNumberGuessingGraph:
    def _I_init__(self, model_name: str = "gpt2"):
        # Initialize Hugging Face model
        try:
            # Use a text generation pipeline
            self.llm = pipeline('text-generation', model=model_name, device='cuda' if torch.cuda.is_available() else 'cpu')
        except Exception as e:
            print(f"LLM Initialization Error: {e}")
            self.llm = None
        
        # Game parameters
        self.secret_number = random.randint(0, 100)
        self.G = nx.DiGraph()
        
        # Prompts for agent reasoning
        self.selector_prompt = """
        A secret number between 0 and 100 has been chosen.
        You must respond with:
        - "HIGHER" if the guess is too low
        - "LOWER" if the guess is too high
        - "CORRECT" if the guess is exactly right
        Current guess is {guess}. The goal is to determine the relationship to the secret number.
        """
        
        self.guesser_prompt = """
        You are trying to guess a secret number between 0 and 100.
        Use logical reasoning to make your guess.
        Current search range is between {lower} and {upper}.
        Your next logical guess is: 
        """
    
    def create_graph_workflow(self) -> nx.DiGraph:
        """
        Create the initial graph workflow for the number guessing game
        """
        # Create graph nodes for different game states
        self.G.add_nodes_from([
            ('start', {'type': 'state', 'description': 'Game initialization'}),
            ('guess_initiated', {'type': 'state', 'description': 'Guess made'}),
            ('feedback_received', {'type': 'state', 'description': 'Feedback processed'}),
            ('game_solved', {'type': 'state', 'description': 'Correct number found'})
        ])
        
        # Add edges representing state transitions
        self.G.add_edges_from([
            ('start', 'guess_initiated', {'action': 'initial_guess'}),
            ('guess_initiated', 'feedback_received', {'action': 'evaluate_guess'}),
            ('feedback_received', 'guess_initiated', {'action': 'adjust_guess'}),
            ('feedback_received', 'game_solved', {'action': 'correct_guess'})
        ])
        
        return self.G
    
    def get_model_feedback(self, prompt: str) -> str:
        """
        Get feedback from the Hugging Face model
        """
        if not self.llm:
            raise ValueError("Language model not initialized")
        
        # Generate response
        response = self.llm(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']
        
        # Extract feedback
        if "HIGHER" in response:
            return "HIGHER"
        elif "LOWER" in response:
            return "LOWER"
        elif "CORRECT" in response:
            return "CORRECT"
        
        # Fallback to binary search logic if model doesn't provide clear feedback
        return "LOWER" if int(prompt.split()[-1]) > self.secret_number else "HIGHER"
    
    def play_game(self, max_attempts: int = 10) -> Dict[str, Any]:
        """
        Execute the number guessing game with graph tracking
        """
        # Create initial graph workflow
        graph = self.create_graph_workflow()
        
        # Game logic variables
        lower_bound = 0
        upper_bound = 100
        attempts = 0
        current_state = 'start'
        
        # Game result tracking
        game_results = {
            'attempts': [],
            'guesses': [],
            'feedbacks': [],
            'success': False
        }
        
        while current_state != 'game_solved' and attempts < max_attempts:
            # Make a guess using binary search
            guess = (lower_bound + upper_bound) // 2
            attempts += 1
            
            # Transition to guess initiated state
            current_state = 'guess_initiated'
            graph.nodes[current_state]['guess'] = guess
            
            # Get model feedback
            try:
                # Prepare prompt for model to get feedback
                feedback_prompt = self.selector_prompt.format(guess=guess)
                feedback = self.get_model_feedback(feedback_prompt)
            except Exception as e:
                print(f"Error getting model feedback: {e}")
                feedback = "CORRECT" if guess == self.secret_number else ("HIGHER" if guess < self.secret_number else "LOWER")
            
            # Adjust search bounds based on feedback
            if feedback == "HIGHER":
                lower_bound = guess + 1
            elif feedback == "LOWER":
                upper_bound = guess - 1
            elif feedback == "CORRECT":
                current_state = 'game_solved'
                game_results['success'] = True
            
            # Track game progress
            game_results['attempts'].append(attempts)
            game_results['guesses'].append(guess)
            game_results['feedbacks'].append(feedback)
            
            # Transition to feedback received state
            graph.nodes['feedback_received']['feedback'] = feedback
            current_state = 'feedback_received'
            
            # Add attempt details to graph
            graph.add_node(f'attempt_{attempts}', 
                           guess=guess, 
                           feedback=feedback, 
                           attempt_number=attempts)
            
            print(f"Attempt {attempts}: Guess {guess}, Feedback: {feedback}")
            
            if current_state == 'game_solved':
                print(f"Game solved in {attempts} attempts!")
                break
        
        # Final game state
        if not game_results['success']:
            print("Failed to guess the number within max attempts!")
        
        return {
            'graph': graph,
            'results': game_results,
            'secret_number': self.secret_number
        }
    
    def analyze_graph(self, graph: nx.DiGraph) -> Dict[str, Any]:
        """
        Perform detailed analysis of the game graph
        """
        analysis = {
            'total_nodes': graph.number_of_nodes(),
            'total_edges': graph.number_of_edges(),
            'node_types': {},
            'edge_actions': {}
        }
        
        # Count node types
        for node, data in graph.nodes(data=True):
            node_type = data.get('type', 'unknown')
            analysis['node_types'][node_type] = analysis['node_types'].get(node_type, 0) + 1
        
        # Count edge actions
        for _, _, data in graph.edges(data=True):
            action = data.get('action', 'unknown')
            analysis['edge_actions'][action] = analysis['edge_actions'].get(action, 0) + 1
        
        return analysis
    
    def visualize_game_graph(self, graph: nx.DiGraph):
        """
        Create a visual representation of the game graph
        """
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(graph, k=0.9, iterations=50)
        
        # Node colors based on type
        node_colors = []
        for node in graph.nodes():
            node_type = graph.nodes[node].get('type', 'unknown')
            if node_type == 'state':
                node_colors.append('lightgreen')
            else:
                node_colors.append('lightblue')
        
        # Draw nodes
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=2000, alpha=0.8)
        nx.draw_networkx_edges(graph, pos, edge_color='gray', arrows=True)
        
        # Node labels
        node_labels = {node: node for node in graph.nodes()}
        nx.draw_networkx_labels(graph, pos, labels=node_labels, font_weight='bold')
        
        # Edge labels
        edge_labels = {(u, v): d.get('action', '') for (u, v, d) in graph.edges(data=True)}
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
        
        plt.title("Number Guessing Game - Interaction Graph")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Example usage
def main():
    # Create game instance with a specific Hugging Face model
    game = AdvancedNumberGuessingGraph(model_name="gpt2")
    
    # Play the game
    game_result = game.play_game(max_attempts=10)
    
    # Analyze the graph
    graph_analysis = game.analyze_graph(game_result['graph'])
    print("\nGraph Analysis:")
    print(graph_analysis)
    
    # Visualize the graph
    game.visualize_game_graph(game_result['graph'])

# Run the game
if __name__ == "__main__":
    main()